In [1]:
import pandas as pd
import numpy as np

x_data = pd.read_csv("train.csv").iloc[:,:-1]
label = pd.read_csv("train.csv").iloc[:,-1]

In [3]:
x_data = x_data.iloc[:,1:]

In [4]:
# 정류장에 노선이 얼마나 지나가는지 구하는 코드
# 정류장 아이디 집합
station_set = set(x_data.station_code)
# 정류장별 몇개의 노선이 지나가는지 dictionary형태로 저장하는 변수
station_count = {station_code:0 for station_code in set(x_data.station_code)}
# 노선이 어떤 정류장을 지나가는지 저장하는 dictionary형태의 변수
bus_route_set  = {route_id:[] for route_id in set(x_data.bus_route_id)}

for i in range(len(x_data)):
    # 각 행의 정보
    step_data = x_data.iloc[i]
    bus_route_set[step_data.bus_route_id].append(step_data.station_code)

bus_route_id_set = set(x_data.bus_route_id.to_list())

for route_name in bus_route_id_set:
    bus_route_set[route_name] = list(set(bus_route_set[route_name]))

for route_name in bus_route_id_set:
    for station_id in station_set:
        if station_id in bus_route_set[route_name]:
            station_count[station_id] += 1

In [5]:
def bus_route_count(x):
    x["how_many_station"] = 0
    bus_route_count = {route_id:len(bus_route_set[route_id])   for route_id in set(bus_route_set)}
    for i in range(len(x)):
        step_data = x.iloc[i]
        x.iloc[i,-1] = bus_route_count[step_data["bus_route_id"]]
        if i%1000 == 0:
            print(i)
    return x

In [6]:
x_data = bus_route_count(x_data)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [8]:
import datetime
def make_date(x_data):
    date = x_data["date"]
    for i in range(len(date)):
        today = date[i].split('-')
        today = datetime.date(int(today[0]),int(today[1]),int(today[2]))
        date[i] = today.weekday()
    x_data["day_of_the_week"] = date
    x_data = x_data.drop("date",axis = 1)
    return x_data

In [9]:
x_data = make_date(x_data)

c:\python\venv\tensorflow\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [10]:
import geopy.distance
coords_jejusi = (33.500770, 126.522761) #제주시의 위도 경도
coords_seoquipo = (33.259429, 126.558217) #서귀포시의 위도 경도
def make_distance(x_data):
    x_data['dis_jejusi'] = [geopy.distance.vincenty((x_data['latitude'].iloc[i],x_data['longitude'].iloc[i]), coords_jejusi).km for i in range(len(x_data))]
    x_data['dis_seoquipo'] = [geopy.distance.vincenty((x_data['latitude'].iloc[i],x_data['longitude'].iloc[i]), coords_seoquipo).km for i in range(len(x_data))]
    return x_data
x_data = make_distance(x_data)

c:\python\venv\tensorflow\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  """
c:\python\venv\tensorflow\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  


In [11]:
def add_station_count(x):
    x["station_count"] = 0
    for i in range(len(x)):
        step_data = x.iloc[i]
        if step_data["station_code"] in list(station_count.keys()):
            x.iloc[i,-1] = station_count[step_data["station_code"]]
        else :
            x.iloc[i,-1] = np.median(np.array(list(station_count.values())), axis = 0)
        if i % 1000 ==0:
            print(i)
    return x
x_data = add_station_count(x_data)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [14]:
def make_station_code(x):
    sights_list = '성산일출봉 섭지코지 함덕해수욕장 협재해수욕장 새별오름 한라산 카멜리아힐 비자림 사려니숲길 천지연폭포 이호테우해수욕장 아쿠아플라넷 용머리해안 우도 휴애리자연생활공원 송악산 만장굴 용눈이오름 곽지해수욕장 쇠소깍 에코랜드 테마파크 중문관광단지 정방폭포 절물자연휴양림 한림공원 세화해수욕장 김녕해수욕장 대포주상절리 용두암 천제연 폭포 가파도 외돌개 금능해수욕장 제주허브동산 제주김녕미로공원 모슬포항 백록담 중문색달해수욕장 월정리해수욕장 이중섭거리 표선해수욕장 한담해안산책로 산방산 거문오름 한라수목원 아부오름 수목원테마파크 제주민속촌 테디베어뮤지엄 헬로키티아일랜드'
    sights_list = set(sights_list.split(' '))
    jeju = '제주'
    station_name = x["station_name"]
    station_name_categories = np.zeros((len(x)),dtype = np.int32)
    for i in range(len(station_name)):
        
        for sight in sights_list:
            if station_name[i].count(sight)>0:
                station_name_categories[i] = 1
                break
            else:
                station_name_categories[i] = 2
        if station_name[i].count('제주') > 0:
            station_name_categories[i] = 0
    x["name_attribute"] = station_name_categories
    x = x.drop("station_name",axis = 1)
    return x
x_data = make_station_code(x_data)

In [ ]:
display(x_data)

In [ ]:
x_data["label"]  = label
x_data.corr()["label"]

In [ ]:
from sklearn.preprocessing import StandardScaler
def make_standard_time(x):
    standart_scalar_data = x.loc[:,'6~7_ride':'11~12_ride']
    standardscaler = StandardScaler()
    x1 = standardscaler.fit_transform(X = standart_scalar_data.iloc[:,0].to_numpy().reshape(-1,1))
    x2 = standardscaler.fit_transform(X = standart_scalar_data.iloc[:,1].to_numpy().reshape(-1,1))
    x3 = standardscaler.fit_transform(X = standart_scalar_data.iloc[:,2].to_numpy().reshape(-1,1))
    x4 = standardscaler.fit_transform(X = standart_scalar_data.iloc[:,3].to_numpy().reshape(-1,1))
    x5 = standardscaler.fit_transform(X = standart_scalar_data.iloc[:,4].to_numpy().reshape(-1,1))
    x6 = standardscaler.fit_transform(X = standart_scalar_data.iloc[:,5].to_numpy().reshape(-1,1))
    standart_scalar_data.iloc[:,0] = x1
    standart_scalar_data.iloc[:,1] = x2
    standart_scalar_data.iloc[:,2] = x3
    standart_scalar_data.iloc[:,3] = x4
    standart_scalar_data.iloc[:,4] = x5
    standart_scalar_data.iloc[:,5] = x6
    display(standart_scalar_data)
    x.loc[:,'6~7_ride':'11~12_ride'] = standart_scalar_data
    x = x.drop(['6~7_takeoff', '7~8_takeoff', '8~9_takeoff',
       '9~10_takeoff', '10~11_takeoff', '11~12_takeoff'], axis = 1)
    return x
x_data = make_standard_time(x_data)
    

In [ ]:
display(x_data)

In [ ]:
from sklearn.preprocessing import StandardScaler
standardscaler = StandardScaler()
x_data["정류장별 지나는 노선수"] =  standardscaler.fit_transform(x_data["정류장별 지나는 노선수"].to_numpy().reshape(-1,1))

In [ ]:
display(x_data)

In [ ]:
x_data.to_csv('scaling.csv')

In [ ]:
def make_in_out(x):
    x['in_out'].loc[x["in_out"] == '시외'] = 1
    x['in_out'].loc[x["in_out"] == '시내'] = 0
    return x

In [ ]:
x_data = make_in_out(x_data)

In [ ]:
new_data = x_data.drop("bus_route_id",axis = 1)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
decision = DecisionTreeRegressor()
decision = decision.fit(new_data,label)

In [ ]:
from sklearn.model_selection import cross_val_score

scores_k5 = cross_val_score(decision,new_data,label,scoring = 'mean_squared_error', cv = 5)
print(scores_k5.mean())

In [56]:
test_data = pd.read_csv("test.csv")

In [72]:
test_data = add_station_count(test_data)
test_data = make_distance(test_data)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


KeyboardInterrupt: 

In [ ]:
test_data = bus_route_count(test_data)
test_data = make_station_code(test_data)
test_data = make_standard_time(test_data)

In [64]:
display(test_data[test_data["station_code"] not in bus_route_set])

TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [63]:
display(test_data)

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,...,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,station_count
0,415423,2019-10-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,4.0,4.0,...,2.0,9.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1
1,415424,2019-10-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,6.0,...,1.0,8.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,71
2,415425,2019-10-01,4270000,시외,432,정존마을,33.48181,126.47352,2.0,4.0,...,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,36
3,415426,2019-10-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.49252,1.0,11.0,...,8.0,26.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,2
4,415427,2019-10-01,4270000,시내,1636,롯데호텔,33.24872,126.41032,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228165,643588,2019-10-16,32820000,시내,786,고산환승정류장(고산1리),33.30073,126.18044,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0
228166,643589,2019-10-16,32820000,시내,1080,애월고등학교,33.46262,126.33447,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
228167,643590,2019-10-16,32820000,시내,1129,한림환승정류장(한림리),33.41437,126.26336,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
228168,643591,2019-10-16,32820000,시내,1564,제주시외버스터미널,33.49946,126.51479,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
